# Asistente Vocacional FPUC — Colab

Este notebook implementa un **asistente vocacional** usando **Azure OpenAI** con:
- Embeddings (`emb-vocatest`) para búsqueda semántica sobre *Binder1.pdf*
- Chat (`phi4mini-chat`) para respuestas contextuales
- Variables seguras desde `.env` con *fallback* a `getpass()`

> **Requisitos previos**  
> 1) Tener un recurso Azure OpenAI con deployments: `phi4mini-chat` y `emb-vocatest`  
> 2) Subir tu archivo `Binder1.pdf` a la carpeta `data/` de este entorno  
> 3) (Opcional) Subir `.env` en la raíz del proyecto con tus credenciales


In [ ]:
# ⬇️ Instalación de dependencias
!pip -q install pdfplumber python-dotenv requests tqdm scikit-learn pandas numpy

In [ ]:
# 🔐 Carga de variables de entorno desde .env y fallback interactivo
import os
from getpass import getpass

try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception as e:
    print("python-dotenv no disponible, se usará solo input interactivo.")

if not os.getenv("AZURE_OPENAI_KEY"):
    os.environ["AZURE_OPENAI_KEY"] = getpass("Introduce AZURE_OPENAI_KEY: ")

if not os.getenv("AZURE_OPENAI_ENDPOINT"):
    os.environ["AZURE_OPENAI_ENDPOINT"] = input("Introduce AZURE_OPENAI_ENDPOINT (ej: https://oai-vocatest.services.ai.azure.com/): ").strip()

if not os.getenv("AZURE_OPENAI_DEPLOYMENT"):
    os.environ["AZURE_OPENAI_DEPLOYMENT"] = input("Introduce AZURE_OPENAI_DEPLOYMENT (ej: phi4mini-chat): ").strip()

if not os.getenv("AZURE_OPENAI_EMBEDDING"):
    os.environ["AZURE_OPENAI_EMBEDDING"] = input("Introduce AZURE_OPENAI_EMBEDDING (ej: emb-vocatest): ").strip()

print("✅ Variables cargadas.")

In [ ]:
# 📚 Importaciones y helpers
import pdfplumber
import re
import json
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

API_KEY = os.environ["AZURE_OPENAI_KEY"]
ENDPOINT = os.environ["AZURE_OPENAI_ENDPOINT"]
CHAT_DEPLOYMENT = os.environ["AZURE_OPENAI_DEPLOYMENT"]
EMB_DEPLOYMENT  = os.environ["AZURE_OPENAI_EMBEDDING"]

CHAT_URL = f"{ENDPOINT}openai/deployments/{CHAT_DEPLOYMENT}/chat/completions?api-version=2024-05-01-preview"
EMB_URL  = f"{ENDPOINT}openai/deployments/{EMB_DEPLOYMENT}/embeddings?api-version=2024-05-01-preview"

HEADERS = {"Content-Type": "application/json", "api-key": API_KEY}

def clean_text(t: str) -> str:
    t = re.sub(r"[\u0000-\u001F]+", " ", t)
    t = re.sub(r"\s+", " ", t)
    return t.strip()

def chunk_text(text: str, max_chars: int = 5000, overlap: int = 500):
    # max_chars ~ aprox 800-1000 tokens según idioma; ajustable
    chunks = []
    i = 0
    while i < len(text):
        chunk = text[i:i+max_chars]
        chunks.append(chunk.strip())
        i += max_chars - overlap
    return [c for c in chunks if c]

In [ ]:
# 📄 Extraer texto desde Binder1.pdf (sube tu archivo a ./data/Binder1.pdf)
from pathlib import Path

pdf_path = Path("data/Binder1.pdf")
assert pdf_path.exists(), "❌ No se encontró data/Binder1.pdf. Sube el archivo a la carpeta data/."

pages_text = []
with pdf_path.open("rb") as f:
    with pdfplumber.open(f) as pdf:
        for page in pdf.pages:
            txt = page.extract_text() or ""
            pages_text.append(txt)

full_text = clean_text("\n".join(pages_text))
print("✅ Texto extraído. Longitud de caracteres:", len(full_text))

# Dividir en chunks
chunks = chunk_text(full_text, max_chars=5000, overlap=500)
print(f"✅ Chunks generados: {len(chunks)}")

In [ ]:
# 🧠 Generar embeddings de todos los chunks (puede tardar unos minutos según largo)
def get_embeddings(text_list):
    resp = requests.post(EMB_URL, headers=HEADERS, data=json.dumps({"input": text_list}))
    if resp.status_code != 200:
        raise RuntimeError(f"Error embeddings: {resp.status_code} {resp.text}")
    data = resp.json()["data"]
    return [d["embedding"] for d in data]

# Procesar por lotes para evitar payloads grandes
batch_size = 16
all_embeddings = []
for i in tqdm(range(0, len(chunks), batch_size), desc="Embedding batches"):
    batch = chunks[i:i+batch_size]
    embs = get_embeddings(batch)
    all_embeddings.extend(embs)

emb_arr = np.array(all_embeddings, dtype=np.float32)
print("✅ Matriz de embeddings:", emb_arr.shape)

# Guardar a disco
out_df = pd.DataFrame({"chunk": chunks, "embedding": list(all_embeddings)})
out_df.to_pickle("binder_embeddings.pkl")
print("💾 Guardado: binder_embeddings.pkl")

In [ ]:
# 🔍 Búsqueda semántica (top-k)
def retrieve_context(query: str, top_k: int = 3):
    r = requests.post(EMB_URL, headers=HEADERS, data=json.dumps({"input": query}))
    if r.status_code != 200:
        raise RuntimeError(f"Error embedding query: {r.status_code} {r.text}")
    q_emb = np.array(r.json()["data"][0]["embedding"], dtype=np.float32).reshape(1, -1)
    db = pd.read_pickle("binder_embeddings.pkl")
    M = np.vstack(db["embedding"].values)
    sims = cosine_similarity(q_emb, M)[0]
    idxs = sims.argsort()[::-1][:top_k]
    hits = db.iloc[idxs]
    # devolver texto concatenado
    context = "\n\n---\n\n".join(hits["chunk"].tolist())
    return context, hits[["chunk"]]

# Pequeña prueba de recuperación
ctx, refs = retrieve_context("¿Qué es la gratuidad y cómo postular?")
print(ctx[:500], "...")

In [ ]:
# 🧾 Prompt institucional (puedes editarlo o cargarlo desde prompt_vocacional.txt)
PROMPT_SISTEMA = """Este asistente está diseñado para ayudar a postulantes a carreras universitarias o técnicas en Chile a tomar decisiones informadas sobre su futuro académico y profesional. Trabaja en colaboración con la Fundación Por Una Carrera y la ONG Innovacien, por lo que su enfoque es inclusivo, confiable, claro y siempre centrado en el bienestar del estudiante.

Proporciona orientación sobre procesos de admisión, requisitos de postulación, becas y beneficios estatales, opciones académicas, vida universitaria y decisiones vocacionales.

El asistente incorpora preguntas frecuentes y actividades de autoconocimiento extraídas del material educativo de Fundación Por Una Carrera (Binder1.pdf) y recursos oficiales del DEMRE/FUAS. Debe citar o recomendar recursos cuando corresponda.

El asistente debe:
- Explicar con claridad y sin tecnicismos innecesarios.
- Adaptar sus respuestas a la realidad chilena actual.
- Mostrar sensibilidad frente a contextos de vulnerabilidad social y educativa.
- Proporcionar enlaces, fuentes oficiales y recursos cuando estén disponibles.
- Promover la toma de decisiones informadas sin imponer opciones.

Debe evitar:
- Reemplazar la orientación profesional presencial.
- Dar consejos cerrados o categóricos que limiten opciones.
- Suponer información sin fundamentos o sin contexto.

Al comenzar, saluda indicando que esta es una iniciativa conjunta entre la Fundación Por Una Carrera (https://www.instagram.com/porunacarrera) e Innovacien (https://www.instagram.com/innovacien), e invita a seguir ambas cuentas en Instagram y a compartir su experiencia con el chat.
"""

In [ ]:
# 💬 Función de respuesta con contexto
def chat_with_context(user_query: str, top_k: int = 3, temperature: float = 0.4, max_tokens: int = 600):
    context, refs = retrieve_context(user_query, top_k=top_k)
    payload = {
        "messages": [
            {"role": "system", "content": PROMPT_SISTEMA},
            {"role": "user", "content": f"Contexto relevante:\n{context}\n\nPregunta del usuario: {user_query}"},
        ],
        "temperature": temperature,
        "max_tokens": max_tokens
    }
    resp = requests.post(CHAT_URL, headers=HEADERS, data=json.dumps(payload))
    if resp.status_code != 200:
        raise RuntimeError(f"Error chat: {resp.status_code} {resp.text}")
    answer = resp.json()["choices"][0]["message"]["content"]
    return answer, refs

# Prueba rápida
ans, refs = chat_with_context("¿Cómo postulo a beneficios estudiantiles del Estado?")
print("🤖 Respuesta:\n", ans[:800], "...")

In [ ]:
# 🧑‍💻 Interfaz simple de conversación (ejecuta esta celda y luego llama a chat_with_context)
q = "Quiero estudiar pedagogía pero no sé si califico para gratuidad. ¿Qué debo revisar?"
ans, refs = chat_with_context(q, top_k=3)
print("🧍 Usuario:", q)
print("\n🤖 Asistente:\n", ans)
print("\n🔎 Fragmentos usados (primeros 200 chars de cada uno):")
for i, ch in enumerate(refs["chunk"].tolist(), 1):
    print(f"[{i}] {ch[:200]}...")